## Platts Fujairah Statistics - Inventories (weekly)
Extracting bunkers sale data from Platts Fujairah Statistics:

URL: https://fujairah.platts.com/fujairah

Requirement for using selenium for automating browser interaction:

install browser driver: https://sites.google.com/a/chromium.org/chromedriver/downloads
have Chrome browser installed
If you have a problem while starting selenium, it's likely your Chrome has been upgraded. Check the link above for new version of the webdriver, put it at ..\drivers and try again.

## Workflow

This notebook performs the following steps:

1. Open the website
1. Select the data points 
1. Download the data points in a dataframe stocks
    

In [14]:
pip install selenium


SyntaxError: invalid syntax (<ipython-input-14-be497f362238>, line 1)

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import datetime
import sqlalchemy
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

### selenium parameters

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome('/Users/LYNGOPOULOU_D/PycharmProjects/scraper/drivers/chromedriver')

In [3]:
def download_data():
    try:

        main_df = pd.DataFrame([])
        url = 'https://fujairah.platts.com/fujairah'
        driver.get(url)
        time.sleep(10)
        table = driver.find_element_by_id("panel-1051-table")
        table_rows = table.find_elements_by_tag_name('tr')
        bunker = []
        for tr in table_rows:
            td = tr.find_elements_by_tag_name('td')
            row = [tr.text for tr in td]
            bunker.append(row)
        panel = driver.find_element_by_id("component-1065")
        converted_date = panel.find_element_by_class_name('date-updated').text
        unit = driver.find_element_by_id("component-1047").text
        driver.close()
    except Exception as e:
        raise e
    return bunker, converted_date, unit 


In [4]:
def transform_data(bunker, converted_date, unit):
    bunker = pd.DataFrame(bunker)
    bunker.columns = ['fuel', 'volume']
    bunker['volume'] = bunker['volume'].str.replace(',', '')
    bunker['volume'] = pd.to_numeric(bunker['volume'])
    bunker['date'] =  pd.to_datetime(converted_date)
    bunker['unit']= unit[unit.find('(')+1:unit.find(')')]
    return bunker


In [5]:
bunker, converted_date, unit = download_data()

In [6]:
bunker_df = transform_data(bunker, converted_date, unit)

In [7]:
bunker_df.head()

,fuel,volume,date,unit
0,180cst Low Sulfur Fuel Oil,1752,2021-08-01,m³
1,380cst Low Sulfur Fuel Oil,541179,2021-08-01,m³
2,380cst Marine Fuel Oil,129410,2021-08-01,m³
3,Marine Gasoil,2215,2021-08-01,m³
4,Low Sulfur Marine Gasoil,28662,2021-08-01,m³


## Testing time

In [15]:
pip install pycountry


  Using cached pycountry-20.7.3.tar.gz (10.1 MB)
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746867 sha256=39483486d842e7bc1628d8f15da75bb9e36b92a027e614ba278b0a35422a3f35
  Stored in directory: c:\users\lyngopoulou_d\appdata\local\pip\cache\wheels\09\eb\0d\4ee773c6a4aadc2a43cb5c1d07f268f13c4cdc0eec88e7c1ef
Successfully built pycountry
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine

In [1]:
cd ..

C:\Users\LYNGOPOULOU_D\PycharmProjects\scraper


In [2]:
 %load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [4]:
 from scraper.core import factory

job = factory.get_scraper_job('com_platts_fujairah', 'platts_fujairah_bunkers',full_load=True)
job.run()

DEBUG:scraper.core.factory:Loading module scraper.jobs.com_platts_fujairah.platts_fujairah_bunkers
DEBUG:scraper.core.factory:Getting class PlattsFujairahBunkersJob
INFO:scraper.core.job:Temporary table name: #com_platts_fujairah_bunkers_temp, final table name: com_platts_fujairah_bunkers_data
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:49982/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\LYNGOPOULOU_D\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\LYNGOPOULOU_D\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-siz

ERROR:scraper.core.job:Exception in update_sources_metadata(): 'BaseSource' object has no attribute 'last_download'
Traceback (most recent call last):
  File "C:\Users\LYNGOPOULOU_D\PycharmProjects\scraper\scraper\core\job.py", line 872, in update_sources_metadata
    data["last_download"] = source.last_download
AttributeError: 'BaseSource' object has no attribute 'last_download'
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta.iea.org:8000
DEBUG:urllib3.connectionpool:http://vipenta.iea.org:8000 "PUT /dimension/source/7085 HTTP/1.1" 201 10
INFO:scraper.core.utils:update_sources_metadata: 52.744388580322266 ms
